## Hint1:  MySQLにPythonからアクセスする方法

In [2]:
import pymysql
import pandas.io.sql as psql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial as sp
import scipy.sparse as sparse
from sklearn.decomposition import NMF, TruncatedSVD #皮膚地、徳一

In [3]:
con = pymysql.connect(host="recommendation-201810.csbsmnjyxb52.ap-northeast-1.rds.amazonaws.com",
                     user="guest1",
                     password="guest1",
                      database="movies",
                     port=3306)

In [4]:
sql = """
SELECT
a.userId,
a.movieId,
c.title,
a.rating
FROM rating as a 
INNER JOIN 
links as b
on a.movieId = b.movieId
INNER JOIN
movies_meta as c 
ON b.tmdbid = c.tmdbid
"""

In [11]:
# keywordにあってmovies_metaにないものを取り出す
sql = """
SELECT
a.tmdbid,
a.keywords,
b.title
FROM keywords as a
LEFT OUTER JOIN
movies_meta as b
on a.tmdbid = b.tmdbid
WHERE b.title is NULL
LIMIT 100
"""

In [14]:
# 個数を調べる
sql = """
SELECT
count(a.tmdbid)

FROM keywords as a
LEFT OUTER JOIN
movies_meta as b
on a.tmdbid = b.tmdbid
WHERE b.title is NULL
LIMIT 100
"""

In [17]:
# 重複を調べる
# Group by の後はhaving
sql = """
SELECT
tmdbid,
count(*)
FROM keywords
GROUP BY tmdbid
HAVING count(*) >=2
"""

In [20]:
sql = """
select
a.tmdbid,
a.keywords,
b.genres,
b.title
from
(select
tmdbid,
max(keywords) as keywords
from movies.keywords
group by tmdbid) as a

INNER JOIN

(select
tmdbid,
max(genres) as genres,
max(title) as title
from movies.movies_meta
group by tmdbid) as b

on a.tmdbid = b.tmdbid
"""

In [21]:
data = psql.read_sql(sql, con)

In [22]:
data.head()

,tmdbid,keywords,genres,title
0,100,"[{'id': 502, 'name': 'ambush'}, {'id': 567, 'n...","[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...","Lock, Stock and Two Smoking Barrels"
1,10000,"[{'id': 1253, 'name': 'roommate'}, {'id': 1499...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",La estrategia del caracol
2,10001,"[{'id': 1815, 'name': 'atomic bomb'}, {'id': 2...","[{'id': 35, 'name': 'Comedy'}, {'id': 878, 'na...",Young Einstein
3,100010,"[{'id': 3203, 'name': 'pilot'}, {'id': 3588, '...","[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",Flight Command
4,100017,"[{'id': 2843, 'name': 'fetishism'}, {'id': 326...","[{'id': 18, 'name': 'Drama'}]",Hounded


In [5]:
data = data.drop_duplicates(subset=['userId', 'title'])

In [6]:
data.head()

,userId,movieId,title,rating
0,7,1,Toy Story,3.0
1,9,1,Toy Story,4.0
2,13,1,Toy Story,5.0
3,15,1,Toy Story,2.0
4,19,1,Toy Story,3.0


In [93]:
data['movieId']= data['movieId'].astype(int) 
movie = data.drop_duplicates(subset=['movieId', 'title'])[['movieId', 'title']].set_index('movieId')
#movie['movieId']= movie['movieId'].astype(int) 
movie.head()

C:\Users\Hiroshi.Hiromax\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,title
movieId,
1,Toy Story
2,Jumanji
3,Grumpier Old Men
4,Waiting to Exhale
5,Father of the Bride Part II


In [81]:
movie_dict = movie.to_dict()['title']

In [117]:
rating_matrix = data.pivot(index='userId', columns='movieId', values='rating')
rating_matrix.fillna(0, inplace=True)

In [118]:
rating_matrix.tail()

movieId,1,10,100,100017,100032,100034,100083,100106,100159,100163,...,99795,998,99811,99813,99839,99846,999,99912,99917,99992
userId,,,,,,,,,,,,,,,,,,,,,
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
title_list = np.array(rating_matrix.columns)
user_list = np.array(rating_matrix.index)

In [122]:
rating_matrix_ar = np.array(rating_matrix)
rating_matrix_ar

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.]])

In [106]:
rating_matrix_ar[1]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [107]:
rating_matrix_ar.shape

(671, 8994)

In [66]:
# User * User

In [67]:
already_followed_topic = \
 np.array(data[data['userId'] == user_list[0]]['title'])
data[data['userId'] == user_list[0]]

,userId,movieId,title,rating
1611,1,31,Dangerous Minds,2.5
24963,1,1029,Dumbo,3.0
25666,1,1061,Sleepers,3.0
27288,1,1129,Escape from New York,2.0
27705,1,1172,Cinema Paradiso,4.0
32372,1,1263,The Deer Hunter,2.0
33548,1,1287,Ben-Hur,2.0
33876,1,1293,Gandhi,2.0
34552,1,1339,Dracula,3.5
34633,1,1343,Cape Fear,2.0


In [94]:
already_followed_topic

array(['Dangerous Minds', 'Dumbo', 'Sleepers', 'Escape from New York',
       'Cinema Paradiso', 'The Deer Hunter', 'Ben-Hur', 'Gandhi',
       'Dracula', 'Cape Fear', 'Star Trek: The Motion Picture',
       'Beavis and Butt-Head Do America', 'The French Connection', 'Tron',
       'The Gods Must Be Crazy', 'Willow', 'Antz', 'The Fly',
       'Time Bandits', 'Blazing Saddles'], dtype=object)

In [68]:
def get_cosine_similarity(x, y):
    return 1- sp.distance.cosine(x, y)

def get_jaccard_similarity(x, y):
    return 1 - sp.distance.jaccard(x, y)

user_similarity = []
target_user_row = rating_matrix_ar[0]
for row in rating_matrix_ar:
    sim = get_jaccard_similarity(target_user_row, row)
    user_similarity.append(sim)
user_similarity = np.array(user_similarity)

In [69]:
target_user_row

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [70]:
topN = 20
idx = user_similarity.argsort()[::-1][1:topN+1]
selected_user_similarity = user_similarity[idx]
selected_rating = rating_matrix_ar[idx]

In [71]:
avg_score = []
for col_idx in range(selected_rating.shape[1]):
    weight_score = sum(selected_rating[:, col_idx] * selected_user_similarity)
    similarity_sum = \
     sum(selected_user_similarity[selected_user_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

In [85]:
recommend_num = 5
counter = 0
for recommended_title in title_list[avg_score.argsort()[::-1]]:
    if recommended_title not in already_followed_topic:
        print(movie_dict[int(recommended_title)])
        counter += 1
        if recommend_num <= counter:
            break

Fargo
The Shawshank Redemption
The Matrix
Return of the Jedi
Ronin


# Item * Item

In [96]:
def get_cosine_similarity(x, y):
    return 1- sp.distance.cosine(x, y)

def get_jaccard_similarity(x, y):
    return 1 - sp.distance.jaccard(x, y)

movie_similarity = []
target_movie_column = rating_matrix_ar[1]
for column in rating_matrix_ar:
    sim = get_jaccard_similarity(target_movie_column, column)
    movie_similarity.append(sim)
movie_similarity = np.array(movie_similarity)

In [95]:
rating_matrix_ar[1]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [108]:
topN = 20
col = movie_similarity.argsort()[::-1][1:topN+1]
selected_movie_similarity = movie_similarity[col]
selected_rating = rating_matrix_ar[col]

In [109]:
avg_score = []
for col_idx in range(selected_rating.shape[0]):
    weight_score = sum(selected_rating[:, col_idx] * selected_movie_similarity)
    similarity_sum = \
     sum(selected_movie_similarity[selected_movie_similarity > 0])
    avg_score.append(weight_score / similarity_sum)
avg_score = np.array(avg_score)

In [110]:
recommend_num = 5
counter = 0
for recommended_title in title_list[avg_score.argsort()[::-1]]:
    if recommended_title not in already_followed_topic:
        print(movie_dict[int(recommended_title)])
        counter += 1
        if recommend_num <= counter:
            break

Toy Story
Sense and Sensibility
GoldenEye
Casino
The American President


## Hint2: エラーと格闘する方法

### 1. try~exceptでエラーが出るデータを確認する

In [23]:
import json

In [24]:
for a in data['keywords']:
    b = a.replace("\'", "\"")
    try:
        json.loads(b)
    except:
        print(b)

[{"id": 459, "name": "sexual obsession"}, {"id": 549, "name": "prostitute"}, {"id": 2316, "name": "women"s sexual identity"}, {"id": 5628, "name": "broker"}, {"id": 5916, "name": "gallery owner"}, {"id": 6373, "name": "sadomasochism"}]
[{"id": 772, "name": "handcuffs"}, {"id": 2038, "name": "love of one"s life"}, {"id": 15090, "name": "police officer"}, {"id": 155546, "name": "female police officer"}, {"id": 157216, "name": "handcuffed to a policewoman"}, {"id": 188656, "name": "friendly fire"}]
[{"id": 2038, "name": "love of one"s life"}, {"id": 2343, "name": "magic"}, {"id": 2594, "name": "magic show"}, {"id": 3071, "name": "prince"}, {"id": 3873, "name": "sorcerer"s apprentice"}, {"id": 4158, "name": "enchantment"}, {"id": 163227, "name": "unicorn"}]
[{"id": 919, "name": "smoking"}, {"id": 10776, "name": "marijuana"}, {"id": 54169, "name": "stoner"}, {"id": 169517, "name": "roach"}, {"id": 169522, "name": "lollipop"}, {"id": 169532, "name": "painting toenails"}, {"id": 169534, "name

[{"id": 1452, "name": "loss of child"}, {"id": 2341, "name": "nanny"}, {"id": 2535, "name": "destroy"}, {"id": 2783, "name": "family"s daily life"}, {"id": 3256, "name": "infant"}, {"id": 3927, "name": "rage and hate"}, {"id": 4914, "name": "malice"}, {"id": 5885, "name": "childlessness"}, {"id": 168386, "name": "child wish"}, {"id": 208349, "name": "child"}]
[{"id": 392, "name": "england"}, {"id": 2784, "name": "mother role"}, {"id": 5424, "name": "women"s prison"}, {"id": 6149, "name": "police"}, {"id": 10226, "name": "neighbor"}, {"id": 10721, "name": "pregnant"}, {"id": 11322, "name": "female protagonist"}, {"id": 17986, "name": "miscarriage"}, {"id": 41246, "name": "tailor"}, {"id": 170273, "name": "fingerprinting"}, {"id": 173490, "name": "unwanted pregnancy"}, {"id": 208611, "name": "1950s"}]
[{"id": 941, "name": "tradition"}, {"id": 1450, "name": "shepherd"}, {"id": 1965, "name": "sandstorm"}, {"id": 2429, "name": "will to survive"}, {"id": 2783, "name": "family"s daily life"},

### 2. https://jsonlint.com/ でなぜエラーが出るのかを確認する

In [25]:
def get_keywords(x):
    b = x.replace("\'","\"")
    result = []
    try:
        c = json.loads(b)
        for e in c:
            result.append(e['name'])
    except:
        result = [""]
    return result

In [28]:
data['keywords_parsed'] = data['keywords'].apply(get_keywords)
data['genres_parsed'] = data['genres'].apply(get_keywords)

In [30]:
data.head(10)

,tmdbid,keywords,genres,title,keywords_parsed,genres_parsed
0,100,"[{'id': 502, 'name': 'ambush'}, {'id': 567, 'n...","[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...","Lock, Stock and Two Smoking Barrels","[ambush, alcohol, shotgun, tea, joint, machism...","[Comedy, Crime]"
1,10000,"[{'id': 1253, 'name': 'roommate'}, {'id': 1499...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",La estrategia del caracol,"[roommate, pastor, squatter, anarchist, house,...","[Comedy, Drama]"
2,10001,"[{'id': 1815, 'name': 'atomic bomb'}, {'id': 2...","[{'id': 35, 'name': 'Comedy'}, {'id': 878, 'na...",Young Einstein,"[atomic bomb, nobel prize, rock, albert einste...","[Comedy, Science Fiction]"
3,100010,"[{'id': 3203, 'name': 'pilot'}, {'id': 3588, '...","[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",Flight Command,"[pilot, navy]","[Drama, War]"
4,100017,"[{'id': 2843, 'name': 'fetishism'}, {'id': 326...","[{'id': 18, 'name': 'Drama'}]",Hounded,"[fetishism, masochism, submissive, older woman...",[Drama]
5,10002,"[{'id': 212, 'name': 'london england'}, {'id':...","[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",Mona Lisa,"[london england, prostitute, ex-detainee, chau...","[Drama, Crime, Romance]"
6,100024,"[{'id': 222018, 'name': 'dangerous side effect...","[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",Bloodwork,"[dangerous side effects, clinical trials, alle...","[Horror, Thriller]"
7,10003,"[{'id': 220, 'name': 'berlin'}, {'id': 2139, '...","[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",The Saint,"[berlin, russia, gas, master thief, the saint]","[Thriller, Action, Romance, Science Fiction, A..."
8,100032,[],"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",The Great Los Angeles Earthquake,[],"[Drama, Action]"
9,100033,"[{'id': 548, 'name': 'countryside'}]","[{'id': 18, 'name': 'Drama'}]",Mr. Thank You,[countryside],[Drama]


In [66]:
def get_jac_sim(content1, content2):
    a = set(data.iloc[content1]['genres_parsed'] + data.iloc[content1]['keywords_parsed'])
    b = set(data.iloc[content2]['genres_parsed'] + data.iloc[content2]['keywords_parsed'])
    jac_sim = len(a.intersection(b)) / len(a.union(b))
    return jac_sim

In [91]:
def contents_match(target_content):
    jac_sim_list = []
    for idx in data.index: #行番号の生成
        jac_sim = get_jac_sim(content1=target_content, content2=idx)
        jac_sim_list.append(jac_sim)
    recommend_idx = np.argpartition(-np.array(jac_sim_list), 6)[1:6]
    
    print('Target contents', data.iloc[target_content]['title'])
    print('Recommended', data.iloc[recommend_idx]['title'])
    

In [93]:
contents_match(target_content=40602) #37137, 40602, 36964

Target contents Harry Potter and the Half-Blood Prince
Recommended 24935                     Return to Halloweentown
40958                       Mister Blot's Academy
38680    Harry Potter and the Philosopher's Stone
4029                     Bedknobs and Broomsticks
18558                             The Magic Sword
Name: title, dtype: object


In [88]:
data[data['tmdbid']=='597']

,tmdbid,keywords,genres,title,keywords_parsed,genres_parsed
36964,597,"[{'id': 2580, 'name': 'shipwreck'}, {'id': 298...","[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",Titanic,"[shipwreck, iceberg, ship, panic, titanic, oce...","[Drama, Romance, Thriller]"


In [68]:
jac_sim_list = []
for idx in data.index: #行番号の生成
    jac_sim = get_jac_sim(content1=0, content2=idx)
    jac_sim_list.append(jac_sim)

In [69]:
import numpy as np

In [75]:
recommend_idx = np.argpartition(-np.array(jac_sim_list), 6)[1:6]

In [78]:
data.iloc[recommend_idx]['title']

28938                                Super Troopers
8645                                       Paulette
22666                                  Contact High
19445    Trailer Park Boys: Countdown to Liquor Day
44422                                    Half Baked
Name: title, dtype: object

In [79]:
data.iloc[0]['title']

'Lock, Stock and Two Smoking Barrels'

In [61]:
b

{'Comedy',
 'Drama',
 'anarchist',
 'bogotá',
 'house',
 'lawyer',
 'pastor',
 'revolutionary',
 'roommate',
 'squatter'}

In [62]:
len(a.union(b))

30

In [63]:
len(a.intersection(b))

1

In [64]:
len(a.intersection(b)) / len(a.union(b))

0.03333333333333333